<a href="https://colab.research.google.com/github/fayeholt/4650-implicit-hate_study/blob/main/%20binary_svm_sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import nltk
import torch
# import shap
!pip install shap
import shap
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

import pandas as pd

clf = svm.SVC(kernel='linear')

name = 'hydrated_hate.csv'
dataset = pd.read_csv(name)

dataset = dataset[dataset.content != "NOT AVAILABLE"]
# dataset = dataset[dataset['class'] != 'not_hate']
dataset.drop('ID', inplace=True, axis=1)
dataset.content = dataset.content.astype(str)


In [ ]:
for index, entry in dataset.iterrows():
    if entry['class'] == 'not_hate':
      dataset.loc[index, 'label'] = 0
    else:
      dataset.loc[index, 'label'] = 1

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dataset['content'], dataset['label'], test_size=0.3)
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Tfidf_vect = TfidfVectorizer(max_features=5000)
# Tfidf_vect.fit(dataset['content'])
# Train_X_Tfidf = Tfidf_vect.transform(Train_X)
# Test_X_Tfidf = Tfidf_vect.transform(Test_X)

vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(Train_X)
test_vectors = vectorizer.transform(Test_X)
vectorizer.vocabulary_

In [ ]:
shap.initjs()

In [ ]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear', probability=True)
t0 = time.time()
classifier_linear.fit(train_vectors, Train_Y)
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(Test_Y, prediction_linear, output_dict=True)
print(report)
# print('negative: ', report['neg'])

In [ ]:
svm_explainer = shap.KernelExplainer(classifier_linear.predict_proba,train_vectors)
svm_shap_values = svm_explainer.shap_values(X=test_vectors[:10], nsamples=10, l1_reg="num_features(4)")


In [ ]:
shap.summary_plot(svm_shap_values, test_vectors)

In [ ]:
shap.force_plot(svm_explainer.expected_value[0], svm_shap_values[0])